In [1]:
# bug incompatibilité dans myenv_clone, ok dans nlp_env
# faire du nettoyage dans les envs un jour …
import os
from ollama import Client

OLLAMA_HOST = os.environ.get("OLLAMA_HOST")
client = Client(host=OLLAMA_HOST)


In [ ]:
import pandas as pd

df = pd.read_csv("df_match.csv")

In [3]:
# # test
# response = client.chat(
#     model="llama3.2",
#     messages=[
#         {
#             "role": "user",
#             "content": "Why is the sky blue?",
#         },
#     ],
# )
# print(response)

model='llama3.2' created_at='2025-06-26T14:35:44.364221377Z' done=True done_reason='stop' total_duration=1291411302 load_duration=14430832 prompt_eval_count=31 prompt_eval_duration=1834207 eval_count=283 eval_duration=1274251929 message=Message(role='assistant', content="The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. Sunlight enters Earth's atmosphere and is made up of different wavelengths (colors) of light.\n2. The shorter (blue) wavelengths are scattered more than the longer (red) wavelengths by the tiny molecules of gases such as nitrogen and oxygen in the atmosphere.\n3. This scattering effect is more pronounced for blue light because it has a smaller wavelength, which means it's more easily deflected by the tiny molecules.\n4. As a result, when sunlight enters our eyes, we see mostly the scattered blue light, making the sky appe

In [11]:
def get_prompt(text):
    return [
        {
            "role": "system",
            "content": (
                "Tu es un expert politique. "
                "Classifie chaque intervention comme 'REPUBLIQUE' ou 'AUTRE'. "
                "Réponds uniquement par un de ces deux mots."
            ),
        },
        {"role": "user", "content": f'Classe cette intervention :\n"{text}"\nLabel:'},
    ]


In [ ]:
from tqdm import tqdm

model = "llama3.1:70b"


def do_predictions(prompt_generator, texts, model):
    """
    Inference avec Ollama : prompt_generator, liste de textes, modèle.
    Affiche la progression avec tqdm (en %).
    """
    results = []
    total = len(texts)

    with tqdm(
        total=total,
        desc="Progress",
        unit="item",
        bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{percentage:3.0f}%]",
    ) as pbar:
        for i, text in texts.items():
            try:
                messages = prompt_generator(text)
                response = client.chat(model=model, messages=messages)
                content = response.get("message", {}).get("content", "").strip()
                results.append(content)
            except Exception as e:
                print(f"Erreur ligne {i} : {e}")
                results.append(None)
            pbar.update(1)

    return results


In [13]:
# test avec un extrait
texts = df["Texte"][:10]  # ou .iloc[:10]

# Appel
r_ollama = do_predictions(get_prompt, texts, model="llama3.1:70b")

# Ajout dans le DataFrame
df.loc[texts.index, "prediction"] = r_ollama

Progress: 100%|██████████ | 10/10 [100%]


In [14]:
df["prediction"]

0       REPUBLIQUE
1       REPUBLIQUE
2            AUTRE
3       REPUBLIQUE
4       REPUBLIQUE
           ...    
4587           nan
4588           nan
4589           nan
4590           nan
4591           nan
Name: prediction, Length: 4592, dtype: object